In [1]:
import h5py
import numpy as np


In [2]:
## 1. IMAGES
h5f = h5py.File('datasets/data_train_images.h5', 'r')
train_images = h5f['data_train_images'][:]
h5f.close()

## 2. LANDMARKS
h5f = h5py.File('datasets/data_train_landmarks.h5', 'r')
train_landmarks = h5f['data_train_landmarks'][:]
h5f.close()

## 3. LABELS
h5f = h5py.File('datasets/data_train_labels.h5', 'r')
train_labels = h5f['data_train_labels'][:]
h5f.close()

## display the size of the different data tables
print("> Images")
print('Number of training images     :', train_images.shape[0])
print('Number of frames per sequence :', train_images.shape[1])
print('Image shape                   :', train_images[0][0].shape)

print("\n> Landmarks")
print('Number of landmarks in a face      :', train_landmarks.shape[2])
print('Number of coordinates per landmark :', train_landmarks.shape[-1])

print("\n> Labels")
print('Number of training labels:', train_labels.shape[0])

> Images
Number of training images     : 486
Number of frames per sequence : 10
Image shape                   : (200, 200)

> Landmarks
Number of landmarks in a face      : 478
Number of coordinates per landmark : 3

> Labels
Number of training labels: 486


In [ ]:
print(train_landmarks[0])
print(np.unique(train_labels))

# PyTorch

In [1]:
import torch

device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [2]:
from pierre import train_model, load_dataset, Model1
from ray.tune.schedulers import ASHAScheduler
from ray import tune

2024-01-29 14:56:44,987	INFO worker.py:1724 -- Started a local Ray instance.


In [3]:

config = {
    "n1": tune.choice(np.arange(1, 4096, 1)),
    "n2": tune.choice(np.arange(1, 4096, 1)),
    "n3": tune.choice(np.arange(1, 4096, 1)),
    "n4": tune.choice(np.arange(1, 4096, 1)),
    "n5": tune.choice(np.arange(1, 1000, 1)),
    "p1": tune.choice(np.linspace(0, 1, 100)),
    "p2": tune.choice(np.linspace(0, 1, 100)),
    "p3": tune.choice(np.linspace(0, 1, 100)),
    "p4": tune.choice(np.linspace(0, 1, 100)),
    "p5": tune.choice(np.linspace(0, 1, 50)),
    "lr": tune.loguniform(1e-5, 1e-2),
    "batch_size": tune.choice([256, 512, 1024, 2048]),
    'model': tune.choice([Model1]),
    'loss_fn': torch.nn.CrossEntropyLoss,
    'loader': load_dataset,
    'max_epoch': 1000,
    'device': device,
    'show_bar': False,
}
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=200,
    grace_period=1,
    reduction_factor=2,
)

# Handles the results
result = tune.run(
    train_model.remote,
    resources_per_trial={"cpu": 4, "gpu": 0},
    config=config,
    num_samples=2,
    scheduler=scheduler,
)

# Find the best config on the accuracy parameter
best_trial = result.get_best_trial("accuracy", "max", "last")
print(f"Best trial config: {best_trial.config}")
print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

NameError: name 'np' is not defined

In [14]:

m = train.remote({
    "lr": 1e-3,
    "batch_size": 256,
    'model': Model1,
    'loss_fn': torch.nn.CrossEntropyLoss,
    'loader': load_dataset
}, device)

print("ASDSSSSSSSSSSSS")
ts, _ = load_dataset(256, device)
pt0 = ts.dataset[0][0]
print(pt0)
print(pt0.shape)
print(pt0.unsqueeze(0).shape)
print(pt0.reshape((3, 10, 478)).shape)
print()
mdl = Model1()
r = mdl.forward(pt0.unsqueeze(0).reshape((1, 10, 3 * 478)))
print(r.shape, r)

ASDSSSSSSSSSSSS
tensor([[ 0.4788,  0.6395, -0.1146,  ...,  0.6768,  0.4188,  0.0223],
        [ 0.4776,  0.6404, -0.1145,  ...,  0.6803,  0.4127,  0.0202],
        [ 0.4806,  0.6420, -0.1137,  ...,  0.6823,  0.4116,  0.0186],
        ...,
        [ 0.4742,  0.6650, -0.1075,  ...,  0.6843,  0.4031,  0.0126],
        [ 0.4777,  0.6602, -0.1075,  ...,  0.6862,  0.4048,  0.0134],
        [ 0.4784,  0.6591, -0.1070,  ...,  0.6858,  0.4056,  0.0133]])
torch.Size([10, 1434])
torch.Size([1, 10, 1434])
torch.Size([3, 10, 478])

torch.Size([1, 6]) tensor([[nan, nan, nan, nan, nan, nan]], grad_fn=<AddmmBackward0>)
